# Plastic Bottle RVM - Integrated System

Sistem terintegrasi untuk deteksi botol plastik menggunakan YOLO dan kontrol Arduino.

## Fitur:
- Deteksi botol plastik real-time
- Kompilasi dan upload kode Arduino otomatis
- Komunikasi serial Python-Arduino
- Sistem poin berdasarkan sensor IR

In [3]:
import subprocess
import os
import time

# Konfigurasi Arduino
ARDUINO_FILE = 'plasticbottle_rvm.ino'
ARDUINO_PORT = '/dev/ttyACM0'
ARDUINO_BOARD = 'arduino:avr:mega'  # Sesuaikan dengan board yang digunakan

def compile_and_upload_arduino():
    """
    Fungsi untuk membaca, mengompilasi, dan mengupload kode Arduino
    """
    try:
        # Cek apakah file arduino.cpp ada
        if not os.path.exists(ARDUINO_FILE):
            print(f"❌ Error: File {ARDUINO_FILE} tidak ditemukan!")
            return False
            
        print(f"📖 Membaca file {ARDUINO_FILE}...")
        
        # Baca isi file arduino.cpp
        with open(ARDUINO_FILE, 'r') as f:
            arduino_code = f.read()
            
        print("✅ File Arduino berhasil dibaca")
        print(f"📝 Ukuran kode: {len(arduino_code)} karakter")
        
        # Tampilkan preview kode (opsional)
        print("\n📋 Preview kode Arduino:")
        lines = arduino_code.split('\n')[:10]  # Tampilkan 10 baris pertama
        for i, line in enumerate(lines, 1):
            print(f"{i:2d}: {line}")
        if len(arduino_code.split('\n')) > 10:
            print("    ... (dan seterusnya)")
            
        # Kompilasi dan upload menggunakan Arduino CLI
        print("\n🔨 Mengompilasi dan mengupload ke Arduino...")
        
        # Perintah compile dan upload
        compile_cmd = [
            'arduino-cli', 'compile', 
            '--fqbn', ARDUINO_BOARD,
            '--upload',
            '--port', ARDUINO_PORT,
            ARDUINO_FILE
        ]
        
        # Jalankan perintah
        result = subprocess.run(compile_cmd, 
                              capture_output=True, 
                              text=True, 
                              timeout=60)
        
        if result.returncode == 0:
            print("✅ Arduino berhasil dikompilasi dan diupload!")
            print("⏱️  Menunggu Arduino restart...")
            time.sleep(3)  # Tunggu Arduino restart
            return True
        else:
            print("❌ Error dalam kompilasi/upload:")
            print(result.stderr)
            return False
            
    except subprocess.TimeoutExpired:
        print("❌ Timeout: Proses compile/upload terlalu lama")
        return False
    except FileNotFoundError:
        print("❌ Error: Arduino CLI tidak ditemukan!")
        print("💡 Install dengan: sudo apt install arduino-cli")
        return False
    except Exception as e:
        print(f"❌ Error tidak terduga: {e}")
        return False

def check_arduino_connection():
    """
    Cek koneksi Arduino
    """
    try:
        import serial
        arduino = serial.Serial(ARDUINO_PORT, 9600, timeout=2)
        time.sleep(1)
        arduino.close()
        print(f"✅ Arduino terhubung di {ARDUINO_PORT}")
        return True
    except:
        print(f"❌ Arduino tidak terhubung di {ARDUINO_PORT}")
        return False

# Jalankan setup Arduino
print("🚀 Memulai setup sistem terintegrasi...")
print("\n1️⃣ Cek koneksi Arduino:")
arduino_connected = check_arduino_connection()

if arduino_connected:
    print("\n2️⃣ Kompilasi dan upload kode Arduino:")
    arduino_uploaded = compile_and_upload_arduino()
    
    if arduino_uploaded:
        print("\n🎉 Setup Arduino selesai! Siap untuk deteksi botol.")
    else:
        print("\n⚠️ Setup Arduino gagal, tapi akan lanjut dengan asumsi kode sudah terupload.")
else:
    print("\n⚠️ Arduino tidak terdeteksi, pastikan kabel terhubung dan driver terinstall.")

print("\n" + "="*50)

🚀 Memulai setup sistem terintegrasi...

1️⃣ Cek koneksi Arduino:
✅ Arduino terhubung di /dev/ttyACM0

2️⃣ Kompilasi dan upload kode Arduino:
📖 Membaca file plasticbottle_rvm.ino...
✅ File Arduino berhasil dibaca
📝 Ukuran kode: 1787 karakter

📋 Preview kode Arduino:
 1: // Pin infrared
 2: const int irPins[] = {A0, A1, A2, A3};
 3: 
 4: // Motor DC pin
 5: const int ENA = 8;
 6: const int IN1 = 6;
 7: const int IN2 = 7;
 8: 
 9: // State
10: String inputString = "";
    ... (dan seterusnya)

🔨 Mengompilasi dan mengupload ke Arduino...
✅ Arduino berhasil dikompilasi dan diupload!
⏱️  Menunggu Arduino restart...

🎉 Setup Arduino selesai! Siap untuk deteksi botol.



In [4]:
import cv2
import time
from ultralytics import YOLO
import serial 
import re

# Load model deteksi
print("🤖 Loading YOLO model...")
model = YOLO('best.pt')
print("✅ Model loaded successfully")

# Koneksi ke Arduino dengan error handling
try:
    print(f"🔌 Connecting to Arduino at {ARDUINO_PORT}...")
    arduino = serial.Serial(ARDUINO_PORT, 9600, timeout=1) 
    time.sleep(2)  # Waktu tunggu koneksi
    print("✅ Arduino connected")
except Exception as e:
    print(f"❌ Error connecting to Arduino: {e}")
    print("⚠️ Continuing without Arduino connection...")
    arduino = None

# Inisialisasi kamera
print("📷 Initializing camera...")
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Cannot open camera")
    exit()

# Set resolusi kamera
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
print("✅ Camera initialized")

detected = False
start_time = time.time()
poin_diperoleh = 0  # Variable untuk menyimpan poin dari Arduino

print("\n🎯 Mendeteksi botol selama 10 detik...")
print("Tekan 'q' untuk keluar")
print("=" * 50)

while time.time() - start_time < 10:
    ret, frame = cap.read()
    if not ret:
        print("Error: Tidak bisa membaca frame dari kamera")
        continue

    # Deteksi botol
    results = model.predict(frame, conf=0.5, verbose=False)
    
    # Gambar hasil deteksi
    annotated_frame = frame.copy()
    
    for result in results:
        boxes = result.boxes
        if boxes is not None:
            for box in boxes:
                # Ambil koordinat bounding box
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                conf = box.conf[0].cpu().numpy()
                cls = int(box.cls[0].cpu().numpy())
                
                # Ambil nama class
                class_name = result.names[cls]
                
                # Cek apakah ini botol plastik
                if "bottle" in class_name.lower() or "plastik" in class_name.lower():
                    detected = True
                    # Gambar bounding box (hijau untuk botol terdeteksi)
                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(annotated_frame, f'{class_name}: {conf:.2f}', 
                              (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Tampilkan status deteksi
    status_text = "BOTOL TERDETEKSI!" if detected else "Mencari botol..."
    status_color = (0, 255, 0) if detected else (0, 0, 255)
    cv2.putText(annotated_frame, status_text, (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2)
    
    # Tampilkan timer
    remaining_time = int(10 - (time.time() - start_time))
    cv2.putText(annotated_frame, f'Waktu: {remaining_time}s', (10, 70), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Tampilkan frame
    cv2.imshow('Deteksi Botol Plastik', annotated_frame)

    # Keluar jika tekan 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Keluar jika botol terdeteksi (opsional)
    if detected:
        print("🎯 Botol terdeteksi! Menunggu 2 detik sebelum melanjutkan...")
        time.sleep(2)  # Beri waktu user melihat hasil
        break

cap.release()
cv2.destroyAllWindows()

# Komunikasi dengan Arduino
if arduino:
    try:
        if detected:
            print("\n✅ Botol terdeteksi, mengirim sinyal ke Arduino...")
            arduino.write(b'BOTOL\n')
        else:
            print("\n❌ Tidak ada botol, mengirim sinyal ke Arduino...")
            arduino.write(b'TIDAK\n')

        # Tunggu respons dari Arduino
        print("⏳ Menunggu respons dari Arduino...")
        timeout_start = time.time()
        
        while time.time() - timeout_start < 10:  # Timeout 10 detik
            if arduino.in_waiting:
                response = arduino.readline().decode().strip()
                print(f"📥 Dari Arduino: {response}")
                
                # Parse poin dari response
                if response.startswith("poin:"):
                    try:
                        poin_diperoleh = int(response.split(":")[1])
                        print(f"🎉 Poin diperoleh: {poin_diperoleh}")
                    except:
                        print("⚠️ Error parsing poin dari Arduino")
                        
                elif response == "SELESAI":
                    print("✅ Proses Arduino selesai")
                    break
                    
            time.sleep(0.1)
        else:
            print("⚠️ Timeout menunggu respons Arduino")
            
        arduino.close()
        
    except Exception as e:
        print(f"❌ Error komunikasi dengan Arduino: {e}")
else:
    print("⚠️ Tidak ada koneksi Arduino")

print("\n" + "="*50)
print("📊 HASIL AKHIR:")
print(f"🎯 Botol terdeteksi: {'Ya' if detected else 'Tidak'}")
if detected and poin_diperoleh > 0:
    print(f"🏆 Poin yang diperoleh: {poin_diperoleh}")
print("🏁 Program selesai.")
print("="*50)


🤖 Loading YOLO model...
✅ Model loaded successfully
🔌 Connecting to Arduino at /dev/ttyACM0...
✅ Arduino connected
📷 Initializing camera...
✅ Camera initialized

🎯 Mendeteksi botol selama 10 detik...
Tekan 'q' untuk keluar
🎯 Botol terdeteksi! Menunggu 2 detik sebelum melanjutkan...

✅ Botol terdeteksi, mengirim sinyal ke Arduino...
⏳ Menunggu respons dari Arduino...
📥 Dari Arduino: poin:5
🎉 Poin diperoleh: 5
📥 Dari Arduino: SELESAI
✅ Proses Arduino selesai

📊 HASIL AKHIR:
🎯 Botol terdeteksi: Ya
🏆 Poin yang diperoleh: 5
🏁 Program selesai.
